In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu
df1 = pd.read_csv(r'BERT_EmbeddingText_classification.csv')
df2 = pd.read_csv(r'stock_news_price.csv')

# Gộp dữ liệu
data = pd.merge(df1, df2, left_on='timestamp', right_on='timestamp', how='inner')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# Chọn các cột để dự đoán
columns_to_forecast = ['close', 'open', 'high', 'low', 'Adj close', 'volume']

# Dự đoán cho từng cột
forecast_results = {}

for column in columns_to_forecast:
    # Huấn luyện mô hình ARIMA trên dữ liệu của cột hiện tại
    model = ARIMA(data[column], order=(5, 1, 0))  # Thay đổi (p,d,q) nếu cần
    model_fit = model.fit()
    
    # Dự đoán cho 1 ngày tiếp theo
    forecast = model_fit.forecast(steps=1)
    
    # Lưu kết quả dự đoán
    forecast_results[column] = forecast.iloc[0]  

# Chuyển đổi kết quả dự đoán thành DataFrame
forecast_df = pd.DataFrame(forecast_results, index=[data.index[-1] + pd.Timedelta(days=1)])

# In kết quả dự đoán
print(forecast_df)
for column in columns_to_forecast:
    # Tạo giá trị thực tế cho ngày tiếp theo từ dữ liệu gốc
    actual_value = data[column].iloc[-1]  # Giá trị cuối cùng trong dữ liệu hiện tại
    
    # Tính MSE và R²
    mse = mean_squared_error([actual_value], forecast_df[column])
    
    print(f"Column: {column}, Mean Squared Error: {mse}")
# Vẽ đồ thị cho từng giá trị (tùy chọn)
for column in columns_to_forecast:
    plt.figure(figsize=(10, 6))
    plt.plot(data.index, data[column], label='Actual ' + column.capitalize(), color='blue')
    plt.axvline(x=data.index[-1], color='gray', linestyle='--', label='Forecast Point')
    plt.scatter(forecast_df.index, forecast_df[column], label='Predicted ' + column.capitalize(), color='red')
    plt.title(f'Actual vs Predicted {column.capitalize()} Price for Next Day')
    plt.xlabel('Date')
    plt.ylabel(column.capitalize())
    plt.legend()
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd

df1 = pd.read_csv(r'BERT_EmbeddingText_classification.csv')
df2 = pd.read_csv(r'stock_news_price.csv')

data = pd.merge(df1, df2, left_on='timestamp', right_on='timestamp', how='inner')
data.to_csv(r'BERT_clustering.csv', index=False)

data['Previous Close 1'] = data['close'].shift(1)
data['Previous Close 2'] = data['close'].shift(2)
data['Previous Close 3'] = data['close'].shift(3)
data.fillna(0, inplace=True)
# Convert 'timestamp' to datetime objects
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Convert 'timestamp' to seconds since epoch
data['timestamp_seconds'] = data['timestamp'].astype('int64') // 10**9  

y = data[['close', 'Adj close']]
# Use 'timestamp_seconds' instead of 'timestamp'
x = data.drop(['close', 'Adj close', 'events', 'timestamp'], axis=1) 

# Convert 'Cluster' column to numeric if it's not already
x['Cluster'] = pd.to_numeric(x['Cluster'], errors='coerce')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

comparision = pd.DataFrame({'Actual_close': y_test['close'], 
                            'Actual_Adj_close': y_test['Adj close'],
                            'Predicted_close': y_pred[:, 0], 
                            'Predicted_Adj_close': y_pred[:, 1]})
print(comparision)

plt.scatter(y_test, y_pred)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
# Get the maximum value for each column in y_test
max_y_test = y_test.max().max()  # Get the maximum of the maximum values in each column

# Get the maximum value for each column in y_pred and then the overall maximum
max_y_pred = y_pred.max()  # Get the maximum value in the array

# Now compare the two maximum values
max_value = max(max_y_test, max_y_pred)
plt.plot([0, max_value], [0, max_value], color='red', linestyle='--', linewidth=2)
plt.title('Actual Prices vs Predicted Prices')
plt.show()